#### [ 데이터 전처리 - 표준화 ]
- 다양한 데이터 수집 시 각각 다른 형태/포맷으로 수집이 되는 경우 발생
- 데이터 단위 불일치 / 자료형 불일치 / 컬럼별 데이터의 범위차이 큰 경우
- 분석을 용이하게 하기 위해서 일치 필요
- 단위 변환/자료형 변환/스케일링

In [40]:
## 모듈 로딩
import pandas as pd

In [41]:
## 데이터 준비
DATA_FILE = '../Data/auto_mpg.csv'

## -> csv 파일 경우 : 첫번째 줄 컬럼명 여부 체크, 데이터 구분자 쉼표(,) 체크
mpgDF = pd.read_csv(DATA_FILE)

## 데이터 확인
mpgDF.info()
display(mpgDF.head(3), mpgDF.describe(include="all")) ## 실제 데이터, 컬럼별 통계 데이터

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
count,398.000000,398.000000,398.000000,398,398.000000,398.000000,398.000000,398.000000,398
unique,NaN,NaN,NaN,94,NaN,NaN,NaN,NaN,305
top,NaN,NaN,NaN,150,NaN,NaN,NaN,NaN,ford pinto
freq,NaN,NaN,NaN,22,NaN,NaN,NaN,NaN,6
mean,23.514573,5.454774,193.425879,NaN,2970.424623,15.568090,76.010050,1.572864,NaN
std,7.815984,1.701004,104.269838,NaN,846.841774,2.757689,3.697627,0.802055,NaN
min,9.000000,3.000000,68.000000,NaN,1613.000000,8.000000,70.000000,1.000000,NaN
25%,17.500000,4.000000,104.250000,NaN,2223.750000,13.825000,73.000000,1.000000,NaN
50%,23.000000,4.000000,148.500000,NaN,2803.500000,15.500000,76.000000,1.000000,NaN
75%,29.000000,8.000000,262.000000,NaN,3608.000000,17.175000,79.000000,2.000000,NaN


[2] 데이터 전처리 <hr>
- 컬럼별 : 고유값 / 결측치 / 중복값 / 이상치 체크 및 처리
- 컬럼별 자료형 : 컬럼별 데이터가 가진 의미 파악 ==> 의미에 맞는 자료형 여부
- 컬럼별 정규화 : 값의 범위 일치

[2-1] 컬럼별 : 고유값 / 결측치 /자료형 변환

In [42]:
## ---------------------------------------------------------
## 함수 기능 : 컬럼별 고유값 개수 및 리스트 출력
## 함수 이름 : print_unique()
## 매개 변수 : DataFrame인스턴스
## 결과 반환 : -
## ---------------------------------------------------------

def print_unique(df):
    for col in df.columns:
        print(f'[{col}] ---- {df[col].nunique()}개')
        print(df[col].unique())

In [43]:
## -------------------------------------------------
## 컬럼별 체크
## ------------------------------------------------
## -> 1) 결측치 체크
mpgDF2 = mpgDF.copy()
mpgDF2.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [44]:
## -> 2) 고유값 체크

## -> horsepower컬럼의 이상 문자 '?' => 결측치 문자로 치환
mpgDF2['horsepower'] = mpgDF2['horsepower'].replace("?", pd.NA)
mpgDF2.isna().sum()

## 6개로 삭제 처리
mpgDF2.dropna(inplace=True)
mpgDF2.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [45]:
## 3) 자료형 변환()
cols = ['horsepower', 'cylinders', 'model year', 'origin']
types = ['int16', 'category', 'category', 'category']

for col, dtype in zip(cols, types):
    mpgDF2[col] = mpgDF2[col].astype(dtype)

print("데이터 타입 ---------\n", mpgDF2.dtypes)

데이터 타입 ---------
 mpg              float64
cylinders       category
displacement     float64
horsepower         int16
weight             int64
acceleration     float64
model year      category
origin          category
car name          object
dtype: object


In [46]:
## 4) 단위 변한 :기존 컬럼 기반 새로운 컬럼 추가
##    mpg컬럼 ==> kpl컬럼 추가 : mpg * 0.425
mpgDF2['kpl'] = mpgDF2['mpg'] * 0.425
mpgDF2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,kpl
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,7.650
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,6.375
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite,7.650
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst,6.800
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino,7.225


In [47]:
## 5) 수치형 => 범주형 변환 처리
## -> 출력 horsepower을 저출력/일반출력/고출력으로 변경
## -> 숫자 -> 번무형
## -> 함수 : pandas.cut()

## 5-1) 구간 분할 개수 지정
print(f"horsepower의 최소값 : {mpgDF2['horsepower'].min()}, 최대값 : {mpgDF2['horsepower'].max()}")
outSR = pd.cut(mpgDF2['horsepower'], 3, right=False, include_lowest=True)
outSR

horsepower의 최소값 : 46, 최대값 : 230


0      [107.333, 168.667)
1      [107.333, 168.667)
2      [107.333, 168.667)
3      [107.333, 168.667)
4      [107.333, 168.667)
              ...        
393       [46.0, 107.333)
394       [46.0, 107.333)
395       [46.0, 107.333)
396       [46.0, 107.333)
397       [46.0, 107.333)
Name: horsepower, Length: 392, dtype: category
Categories (3, interval[float64, left]): [[46.0, 107.333) < [107.333, 168.667) < [168.667, 230.184)]

In [48]:
## 구간 
outSR, retbins = pd.cut(mpgDF2['horsepower'], 3, right = 3, include_lowest=True, retbins = True)
print(outSR)
print(retbins)

0                 (107.333, 168.667]
1                 (107.333, 168.667]
2                 (107.333, 168.667]
3                 (107.333, 168.667]
4                 (107.333, 168.667]
                   ...              
393    (45.815000000000005, 107.333]
394    (45.815000000000005, 107.333]
395    (45.815000000000005, 107.333]
396    (45.815000000000005, 107.333]
397    (45.815000000000005, 107.333]
Name: horsepower, Length: 392, dtype: category
Categories (3, interval[float64, right]): [(45.815000000000005, 107.333] < (107.333, 168.667] < (168.667, 230.0]]
[ 45.816      107.33333333 168.66666667 230.        ]


In [49]:
## 구간변환 Series와 구간 범위값 retbins=True, 구간별 라벨 labels = []
outSR, retbins = pd.cut(mpgDF2['horsepower'], 3, right = 3, include_lowest=True, retbins = True,
                        labels = ['저출력', '보통출력', '고출력'])

# print(outSR)
# print(retbins)


##  변환된 데이터 저장
mpgDF2['horespwer'] = outSR
mpgDF2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,kpl,horespwer
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,7.650,보통출력
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,6.375,보통출력
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite,7.650,보통출력
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst,6.800,보통출력
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino,7.225,보통출력


In [51]:
## origin컬럼의 수치값 ==> 글자값
# mpgDF2.origin.replace({1:'USA', 2:'JPN', 3:'EU'})

mpgDF2.origin = mpgDF2.origin.cat.rename_categories({1:'USA', 2:'JPN', 3:'EU'})
mpgDF2

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,kpl,horespwer
0,18.0,8,307.0,130,3504,12.0,70,USA,chevrolet chevelle malibu,7.650,보통출력
1,15.0,8,350.0,165,3693,11.5,70,USA,buick skylark 320,6.375,보통출력
2,18.0,8,318.0,150,3436,11.0,70,USA,plymouth satellite,7.650,보통출력
3,16.0,8,304.0,150,3433,12.0,70,USA,amc rebel sst,6.800,보통출력
4,17.0,8,302.0,140,3449,10.5,70,USA,ford torino,7.225,보통출력
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,USA,ford mustang gl,11.475,저출력
394,44.0,4,97.0,52,2130,24.6,82,JPN,vw pickup,18.700,저출력
395,32.0,4,135.0,84,2295,11.6,82,USA,dodge rampage,13.600,저출력
396,28.0,4,120.0,79,2625,18.6,82,USA,ford ranger,11.900,저출력
